In [1]:
from google.colab import files

files.upload()

Saving More_cleaned.csv to More_cleaned (2).csv


{'More_cleaned (2).csv': b'text,label\nKunjungan Prabowo ini untuk meresmikan dan menyerahkan proyek bantuan air bersih di lima titik #IndonesiaSentris #IndonesiaHijau #02Melanjutkan #AnakMudaIndonesiaEmas Prabowo Subianto,SumberDayaAlam\nAnies dapat tepuk tangan meriah saat jadi Rektor mewajibkan mata kuliah anti korupsi untuk memutus mata rantai korupsi #AminMiskinkanKoruptor HTTPURL,Politik\nUSERMENTION memang benar sih pendukung 01 ada yang bodoh begitu juga dg pendukung 02 hanya saja menurut pak Ridwan Kamil skemanya terbalik kalau 01 MAYORITAS PENDIDIKAN MENENGAH ATAS artinya ada juga pendidikan rendah yang milih,Demografi\nUSERMENTION Sewaktu anies bersikap kritis ke kinerja pak prabowo dianggap engga sopan karena dianggap kurang menghormati orang tua giliran skrg gibran yang menyebalkan dan sok kritis bahkan dianggap kritis dan keras Apakah ini tidak standar ganda,Politik\nAnies Baswedan Harap ASN termasuk TNI dan Polri Pegang Sumpahnya dalam Pemilu HTTPURL,Politik\nDuh jangan 

In [2]:
pip install sastrawi

In [51]:

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [2]:
import pandas as pd
import numpy as np
import re
import string

df = pd.read_csv('More_cleaned.csv')

In [3]:
df.head()

,text,label
0,Kunjungan Prabowo ini untuk meresmikan dan men...,SumberDayaAlam
1,Anies dapat tepuk tangan meriah saat jadi Rekt...,Politik
2,USERMENTION memang benar sih pendukung 01 ada ...,Demografi
3,USERMENTION Sewaktu anies bersikap kritis ke k...,Politik
4,Anies Baswedan Harap ASN termasuk TNI dan Polr...,Politik


In [4]:
df['label'].value_counts()

label
Politik                  2861
Ekonomi                   591
SumberDayaAlam            576
Ideologi                  548
SosialBudaya              548
PertahanandanKeamanan     547
Demografi                 525
Geografi                  502
Name: count, dtype: int64

In [5]:

# Identify the label to undersample
label_to_undersample = 'Politik'
undersample_fraction = 0.5  # Fraction of the label to keep (e.g., 50%)

# Split the dataset into the label to undersample and the rest
df_label = df[df['label'] == label_to_undersample]
df_other = df[df['label'] != label_to_undersample]

# Randomly undersample the specific label
df_label_undersampled = df_label.sample(frac=undersample_fraction, random_state=42)

# Combine the undersampled data with the rest of the dataset
df_undersampled = pd.concat([df_label_undersampled, df_other])

# Shuffle the combined dataset
df_undersampled = df_undersampled.sample(frac=1, random_state=42).reset_index(drop=True)

# Save or use the undersampled dataset
df_undersampled.to_csv('undersampled_dataset.csv', index=False)

# Print the class distribution
print(df_undersampled['label'].value_counts())


label
Politik                  1430
Ekonomi                   591
SumberDayaAlam            576
SosialBudaya              548
Ideologi                  548
PertahanandanKeamanan     547
Demografi                 525
Geografi                  502
Name: count, dtype: int64


In [6]:
dft = df_undersampled

In [7]:
dft.head()

,text,label
0,Kalau peta politik lihat siapa yang dulu bersa...,Geografi
1,PENGGUNAAN Desa saya dekat pegunungan tinggi p...,Geografi
2,Dilansir dari video yang diposting pada laman ...,Politik
3,PENGGUNAAN Pejabat militer kembali ke penjara ...,PertahanandanKeamanan
4,Apakah Anies tidak takut dilabeli intoleran ke...,SumberDayaAlam


In [1]:
import json

with open('/content/hashtag mapping.txt', 'r', encoding='utf-8') as file:
        hashtag_mapping = json.load(file)

In [2]:
import re
import string
import pandas as pd
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import json

class DataCleaning:
    # Initialization
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    kamus_alay1 = pd.read_csv('https://raw.githubusercontent.com/okkyibrohim/id-abusive-language-detection/master/kamusalay.csv')
    kamus_alay1 = kamus_alay1.set_index('ajh')
    kamus_alay2 = pd.read_csv('https://raw.githubusercontent.com/nasalsabila/kamus-alay/master/colloquial-indonesian-lexicon.csv')
    kamus_alay2 = kamus_alay2.filter(['slang', 'formal'], axis=1)
    kamus_alay2 = kamus_alay2.drop_duplicates(subset=['slang'], keep='first')
    kamus_alay2 = kamus_alay2.set_index('slang')
    stopword1 = list(pd.read_csv('https://raw.githubusercontent.com/datascienceid/stopwords-bahasa-indonesia/master/stopwords_id_satya.txt', header=None)[0])
    custom_word = []
    with open('/content/hashtag mapping.txt', 'r', encoding='utf-8') as file:
        hashtag_mapping = json.load(file)

    @classmethod
    def CleanDataFrame(cls, df, col_name, label_name=None, jum_minimum=None, minimum_kata=0, label_mapping=None, dropna=False):
        final_list_clean = []
        final_list_kotor = []
        final_label = []

        if jum_minimum is None:
            jum_minimum = len(df)
        if len(df) < jum_minimum:
            raise Exception("Jumlah Data Yang Diinginkan melebihi Data yang Ada")
        i = 0
        current = 0

        while i < len(df):
            current_kalimat = df.loc[i][col_name]
            if label_name is not None:
                current_label = df.loc[i][label_name]

            clean_kalimat = cls.__cleanSentence__(current_kalimat)
            if type(clean_kalimat) != str or clean_kalimat is None or clean_kalimat == "":
                print("Ditemukan string kosong setelah di preprocessed pada kalimat: ", current_kalimat)
                clean_kalimat = "NaN"
            if len(clean_kalimat.split(' ')) >= minimum_kata:
                final_list_clean.append(str(clean_kalimat))
                final_list_kotor.append(str(current_kalimat))
                if label_name is not None:
                    if label_mapping is not None:
                        final_label.append(label_mapping[current_label])
                    else:
                        final_label.append(current_label)
                current += 1

                if current % 250 == 0:
                    print("Memproses {} data".format(current))

            if current == jum_minimum:
                break

            i += 1

        data = {
            'raw': final_list_kotor,
            'processed': final_list_clean
        }

        if label_name is not None:
            data['label'] = final_label

        final_df = pd.DataFrame(data)
        if dropna:
            print("NaN Dropped")
            final_df = final_df.dropna(how='any')

        final_df['processed'] = final_df['processed'].astype(str)
        final_df['raw'] = final_df['raw'].astype(str)
        return final_df

    @classmethod
    def __cleanSentence__(cls, text):
        '''
        Melakukan prapemrosesan pada suatu kalimat dengan menghilangkan formatting pada kalimat,
        menghilangkan stopword pada kalimat, mengganti kata alay yang sudah terdefinisikan, serta
        melakukan stemming kalimat tersebut.
        '''

        # Cleaning Formatted Text Link And Tag (@username) using Regex
        text = re.sub('<.*?>', '', text)

        temp_text = list(text)
        for i in range(len(temp_text)):
            if temp_text[i] in string.punctuation:
                temp_text[i] = " "

        text = ''.join(temp_text)

        # Replace hashtags with their mappings
        words = text.split()
        for i in range(len(words)):
            if words[i] in cls.hashtag_mapping:
                words[i] = cls.hashtag_mapping[words[i]]
        text = ' '.join(words)

        text = re.sub('[^a-zA-Z]', ' ', text)
        text = re.sub("\n", " ", text)
        text = text.lower()
        text = re.sub(r'http\S+', 'HTTPURL', text)
        text = re.sub(r'@[a-zA-Z0-9+=\/]*', 'USERMENTION', text)
        text = re.sub("(username|user|url|rt|xf|fx|xe|xa)\s|\s(user|url|rt|xf|fx|xe|xa)", "", text)
        text = re.sub(r'(\w)(\1{2,})', r"\1", text)
        text = re.sub(r"\b[a-zA-Z]\b", "", text)
        text = re.sub('(s{2,})', ' ', text)
        text = ' '.join(text.split())

        text_split = text.split(' ')

        # Mengganti kata-kata yang tidak baku
        for i in range(len(text_split)):
            if text_split[i] in cls.kamus_alay1.index:
                text_split[i] = cls.kamus_alay1.loc[text_split[i]]['saja']
            elif text_split[i] in cls.kamus_alay2.index:
                text_split[i] = cls.kamus_alay2.loc[text_split[i]]['formal']

        # Ensure all elements in text_split are strings before joining
        text_split = [str(word) for word in text_split]

        # Stemming
        stemmed_text = cls.stemmer.stem(' '.join(text_split))

        # Removing Stopwords and custom word
        temp_text_split = []
        for i in range(len(text_split)):
            if (text_split[i] not in cls.stopword1) and (text_split[i] not in cls.custom_word) and (type(text_split[i]) == str):
                temp_text_split.append(text_split[i])
        if len(temp_text_split) == 0:
            return ""
        else:
            final_text = ' '.join(temp_text_split)

        return final_text


In [3]:
df_testing= pd.read_csv('test_data.csv', delimiter=';')
df_testing.head()

,IDText,Text
0,TXT0001,Lu mau org2 pro-demokrasi di negara ini bisa p...
1,TXT0002,Prabowo ditanya soal hutang luar negeri dia me...
2,TXT0003,kiki_daliyo Ganjar Pranowo itulah beliau soso...
3,TXT0004,@kumparan Prabowo Gibran yang bisa melakukan i...
4,TXT0005,@sniperruben45 @uda_zulhendra @ainunnajib Lah ...


In [61]:
# df_testing = df_testing.drop(columns=['IDText'])

In [4]:
df_testing= DataCleaning.CleanDataFrame(df_testing, 'Text',None, None, 2, None, dropna=True)
print(len(df_testing))

Memproses 250 data
Memproses 500 data
Memproses 750 data
Memproses 1000 data
NaN Dropped
1000


In [67]:
df_testing.to_csv('data_test.csv', index=False, encoding="", na_rep="")

In [46]:
dft.head()

,raw,processed,label
0,Kalau peta politik lihat siapa yang dulu bersa...,peta politik lihat pak anis pak prab pak anis ...,Geografi
1,PENGGUNAAN Desa saya dekat pegunungan tinggi p...,penggunaan desa pegunungan tinggi petani menju...,Geografi
2,Dilansir dari video yang diposting pada laman ...,dilansir video diposting laman calon presiden ...,Politik
3,PENGGUNAAN Pejabat militer kembali ke penjara ...,penggunaan pejabat militer kembali penjara unt...,PertahanandanKeamanan
4,Apakah Anies tidak takut dilabeli intoleran ke...,anies takut dilabeli intoleran datang pernikah...,SumberDayaAlam


In [20]:
dft = DataCleaning.CleanDataFrame(dft, 'text', 'label', None, 2, None, dropna=True)
print(len(dft))

Memproses 250 data
Memproses 500 data
Memproses 750 data
Memproses 1000 data
Memproses 1250 data
Memproses 1500 data
Memproses 1750 data
Memproses 2000 data
Memproses 2250 data
Memproses 2500 data
Memproses 2750 data
Memproses 3000 data
Memproses 3250 data
Memproses 3500 data
Memproses 3750 data
Memproses 4000 data
Memproses 4250 data
Memproses 4500 data
Memproses 4750 data
Memproses 5000 data
Memproses 5250 data
NaN Dropped
5256


In [21]:
dft.head(500)

,raw,processed,label
0,Kalau peta politik lihat siapa yang dulu bersa...,peta politik lihat pak anis pak prab pak anis ...,Geografi
1,PENGGUNAAN Desa saya dekat pegunungan tinggi p...,penggunaan desa pegunungan tinggi petani menju...,Geografi
2,Dilansir dari video yang diposting pada laman ...,dilansir video diposting laman calon presiden ...,Politik
3,PENGGUNAAN Pejabat militer kembali ke penjara ...,penggunaan pejabat militer kembali penjara unt...,PertahanandanKeamanan
4,Apakah Anies tidak takut dilabeli intoleran ke...,anies takut dilabeli intoleran datang pernikah...,SumberDayaAlam
...,...,...,...
495,"PENGGUNAAN Bagi saya, ini yang membuat program...",penggunaan membuat program prabowo katakan gil...,SosialBudaya
496,PEMAKAIAN Memang eco park yang dibuat oleh Pak...,pemakaian eco park dibuat pak anies asyik bang...,PertahanandanKeamanan
497,Repost Love dan Share di media sosial teman te...,repost love share media sosial teman teman ful...,Politik
498,7 Politik dinasti yang dimanfaatkan untuk mend...,politik dinasti dimanfaatkan untuk mendukung l...,Politik


In [22]:
dft.to_csv('cleaned_data_5.csv', index=False, encoding="", na_rep="")
dft[dft['processed'] == "NaN"]


,raw,processed,label


In [18]:
dft.head(500)

,raw,processed,label
0,Kalau peta politik lihat siapa yang dulu bersa...,peta politik lihat pak anis pak prab pak anis ...,Geografi
1,PENGGUNAAN Desa saya dekat pegunungan tinggi p...,penggunaan desa pegunungan tinggi petani menju...,Geografi
2,Dilansir dari video yang diposting pada laman ...,dilansir video diposting laman calon presiden ...,Politik
3,PENGGUNAAN Pejabat militer kembali ke penjara ...,penggunaan pejabat militer kembali penjara unt...,PertahanandanKeamanan
4,Apakah Anies tidak takut dilabeli intoleran ke...,anies takut dilabeli intoleran datang pernikah...,SumberDayaAlam
...,...,...,...
495,"PENGGUNAAN Bagi saya, ini yang membuat program...",penggunaan membuat program prabowo katakan gil...,SosialBudaya
496,PEMAKAIAN Memang eco park yang dibuat oleh Pak...,pemakaian eco park dibuat pak anies asyik bang...,PertahanandanKeamanan
497,Repost Love dan Share di media sosial teman te...,repost love share media sosial teman teman ful...,Politik
498,7 Politik dinasti yang dimanfaatkan untuk mend...,politik dinasti dimanfaatkan untuk mendukung l...,Politik
